In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate


  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-9qhp7ey0
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-9qhp7ey0
  Resolved https://github.com/ibm-granite-community/utils to commit 1514191fbbc4605ed4fdfdcb448f2ee41477058f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

REPLICATE_API_TOKEN loaded from Google Colab secret.


In [ ]:
# Build prompt using few-shot format
def fewshot_prompt(context, question, examples):
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output (HTML & JavaScript):
        {example['output']}
        """
        for i, example in enumerate(examples)
    )

    prompt = f"""
You are a senior frontend developer who creates interactive web games using HTML, CSS, and JavaScript only.

Here are some examples of similar tasks you have completed:
{formatted_examples}

Now, generate the code for this new task.

Context: {context}
User Question: {question}

Make sure:
- The HTML structure is clean and semantic.
- JavaScript handles logic for player's choice, random computer choice, winner determination, and score update.
- Styling can be minimal or embedded (inline or <style> block).
- Output ONLY the complete HTML and JavaScript code.
"""
    return prompt

# Main function to get output from Granite model
def get_answer_using_fewshot(context, question, examples):
    prompt = fewshot_prompt(context, question, examples)
    result = model.invoke(prompt)
    return result


In [ ]:
examples = [
{
    "question": "Create a simple coin flip game with heads/tails and result display.",
    "context": "Web game using JavaScript only, no backend. Show flip result and a restart button.",
    "output": """
<!DOCTYPE html>
<html>
<head><title>Coin Flip</title></head>
<body>
  <h1>Coin Flip Game</h1>
  <button onclick="flipCoin()">Flip Coin</button>
  <p id="result"></p>
  <script>
    function flipCoin() {
      const result = Math.random() < 0.5 ? 'Heads' : 'Tails';
      document.getElementById('result').textContent = 'Result: ' + result;
    }
  </script>
</body>
</html>
        """
},
{
    "question": "Create a dice roll game. Show two dice images that roll on click, with simple animation. Show the winner based on the dice values.",
    "context": "A web-based dice game using HTML, CSS, and JavaScript. Two players each roll a die. Animate the dice using CSS, and determine winner based on the higher number.",
    "output": """
<!DOCTYPE html>
<html>
<head>
  <title>Dice Roll Game</title>
  <style>
    body {
      font-family: sans-serif;
      text-align: center;
      padding: 30px;
    }
    .dice {
      font-size: 100px;
      animation: roll 0.5s ease-in-out;
    }
    @keyframes roll {
      0% { transform: rotate(0deg); }
      50% { transform: rotate(180deg); }
      100% { transform: rotate(360deg); }
    }
    button {
      padding: 10px 20px;
      margin-top: 20px;
      font-size: 16px;
    }
  </style>
</head>
<body>
  <h1>Dice Roll Game</h1>
  <div id="player1" class="dice">🎲</div>
  <div id="player2" class="dice">🎲</div>
  <button onclick="rollDice()">Roll</button>
  <h2 id="result"></h2>

  <script>
    function rollDice() {
      const dice = ['⚀','⚁','⚂','⚃','⚄','⚅'];
      const roll1 = Math.floor(Math.random()*6);
      const roll2 = Math.floor(Math.random()*6);

      document.getElementById("player1").textContent = dice[roll1];
      document.getElementById("player2").textContent = dice[roll2];

      const resultText = roll1 > roll2 ? "Player 1 Wins!" : roll1 < roll2 ? "Player 2 Wins!" : "It's a Draw!";
      document.getElementById("result").textContent = resultText;
    }
  </script>
</body>
</html>
    """
},
{
    "question": "Create a reaction time game where the user clicks a colored box as fast as possible when it changes color.",
    "context": "Web-based reaction game with CSS animations and JavaScript timers. A colored box changes randomly after 2–5 seconds. The player must click as fast as possible after the color changes. Show reaction time in ms.",
    "output": """
<!DOCTYPE html>
<html>
<head>
  <title>Reaction Time Game</title>
  <style>
    #box {
      width: 200px;
      height: 200px;
      background-color: red;
      margin: 50px auto;
      display: none;
      transition: background-color 0.2s ease;
    }
    button {
      display: block;
      margin: 20px auto;
      padding: 10px 20px;
    }
  </style>
</head>
<body>
  <h1>Reaction Time Game</h1>
  <p id="message">Click start to begin!</p>
  <div id="box" onclick="recordReaction()"></div>
  <button onclick="startGame()">Start</button>

  <script>
    let start, timeout;

    function startGame() {
      document.getElementById("box").style.display = "none";
      document.getElementById("message").textContent = "Wait for green...";
      const delay = Math.random() * 3000 + 2000;

      timeout = setTimeout(() => {
        document.getElementById("box").style.backgroundColor = "green";
        document.getElementById("box").style.display = "block";
        start = new Date().getTime();
      }, delay);
    }

    function recordReaction() {
      const end = new Date().getTime();
      const reactionTime = end - start;
      document.getElementById("message").textContent = "Your time: " + reactionTime + " ms";
      document.getElementById("box").style.display = "none";
      clearTimeout(timeout);
    }
  </script>
</body>
</html>
    """
}
]


In [ ]:
context = """
Build a Rock Paper Scissors game using HTML, CSS, and JavaScript.
It should have 3 buttons: Rock, Paper, Scissors.
When the user clicks a button, the computer makes a random choice, and the result is shown on the screen.
Score for both sides is tracked.

The layout should include:
- A large centered header with the text "Rock Paper Scissors"
- A scoreboard showing: Player 0 - 0 Computer
- Three emoji buttons for player choice: ✊ Rock, ✋ Paper, ✌️ Scissors
- When clicked, the computer randomly selects a move
- Below the scoreboard, show:
  - Player's choice (emoji + text)
  - In the middle, show result: "Win", "Lose", or "Draw"
  - Computer's choice (emoji + text)

For example:
[Player] 👊 Rock | Win | ✌️ Scissors [Computer]

- Update score dynamically
"""



question = """
Create a fully working Rock Paper Scissors game webpage with scoreboard, emoji buttons, result display.
Use only HTML, CSS, and JavaScript. Output the full code.
"""


result = get_answer_using_fewshot(context, question, examples)
print(f"Generated Code:\n{result}")

Generated Code:
```html
<!DOCTYPE html>
<html>
<head>
  <title>Rock Paper Scissors</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      text-align: center;
      padding: 20px;
    }
    #game-container {
      margin-top: 30px;
    }
    .score {
      font-size: 24px;
      margin-bottom: 10px;
    }
    .choice {
      display: inline-block;
      margin: 0 10px;
      padding: 10px 20px;
      cursor: pointer;
    }
    .result {
      font-size: 20px;
      margin-top: 20px;
    }
  </style>
</head>
<body>
  <h1>Rock Paper Scissors</h1>
  <div id="game-container">
    <div class="score">Player: 0 - Computer: 0</div>
    <div class="choice" data-choice="✊">Rock</div>
    <div class="choice" data-choice="✋">Paper</div>
    <div class="choice" data-choice="✌️">Scissors</div>
  </div>
  <div class="result" id="result"></div>

  <script>
    const choices = ['✊', '✋', '✌️'];
    const choiceElements = document.querySelectorAll('.choice');
    let playerScore = 0;
  

Generated Code:
```html
<!DOCTYPE html>
<html>
<head>
  <title>Rock Paper Scissors</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      text-align: center;
      padding: 20px;
    }
    #game-container {
      margin-top: 30px;
    }
    .score {
      font-size: 24px;
      margin-bottom: 10px;
    }
    .choice {
      display: inline-block;
      margin: 0 10px;
      padding: 10px 20px;
      cursor: pointer;
    }
    .result {
      font-size: 20px;
      margin-top: 20px;
    }
  </style>
</head>
<body>
  <h1>Rock Paper Scissors</h1>
  <div id="game-container">
    <div class="score">Player: 0 - Computer: 0</div>
    <div class="choice" data-choice="✊">Rock</div>
    <div class="choice" data-choice="✋">Paper</div>
    <div class="choice" data-choice="✌️">Scissors</div>
  </div>
  <div class="result" id="result"></div>

  <script>
    const choices = ['✊', '✋', '✌️'];
    const choiceElements = document.querySelectorAll('.choice');
    let playerScore = 0;
    let computerScore = 0;

    choiceElements.forEach(choice => {
      choice.addEventListener('click', () => {
        const playerChoice = choice.dataset.choice;
        playRound(playerChoice);
      });
    });

    function playRound(playerChoice) {
      const computerChoice = getComputerChoice();
      document.querySelector('.result').textContent = `You chose ${playerChoice} | ${getResult(playerChoice, computerChoice)} | Computer chose ${computerChoice}`;

      updateScore(playerChoice, computerChoice);
    }

    function getComputerChoice() {
      return choices[Math.floor(Math.random() * choices.length)];
    }

    function getResult(player, computer) {
      if (player === computer) return "Draw";
      if ((player === '✊' && computer === '✋') || (player === '✋' && computer === '✌️') || (player === '✌️' && computer === '✊')) return "Win";
      return "Lose";
    }

    function updateScore(player, computer) {
      const playerElement = document.querySelector(`[data-choice="${player}"]`);
      const computerElement = document.querySelector(`[data-choice="${computer}"]`);

      if (getResult(player, computer) === "Win") {
        playerScore++;
      } else if (getResult(player, computer) === "Lose") {
        computerScore++;
      }

      document.querySelector('.score').textContent = `Player: ${playerScore} - Computer: ${computerScore}`;
      playerElement.classList.add('highlight');
      computerElement.classList.remove('highlight');

      setTimeout(() => {
        playerElement.classList.remove('highlight');
      }, 500);
    }
  </script>
</body>
</html>
```

In [ ]:
def refine_output_with_fewshot(result, context, question, examples):
    # Format the examples for the prompt
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output (HTML & JavaScript):
        {example['output']}
        """
        for i, example in enumerate(examples)
    )

    prompt = f"""
              You are a senior frontend developer specialized in optimizing HTML, CSS, and JavaScript code.

              You previously generated the following basic web application code:
              {result}

              Now, based on the new request, refine and enhance the code above. Maintain existing functionality, and only apply necessary changes related to the current goal.

              Here are some examples of tasks you’ve refined before:
              {formatted_examples}

              Context:
              {context}

              User Question:
              {question}

              Please output ONLY the full refined HTML and JavaScript code.
              """
    return model.invoke(prompt)


In [ ]:
examples = [
{
    "question": "Create a simple coin flip game with heads/tails and result display.",
    "context": "Web game using JavaScript only, no backend. Show flip result and a restart button.",
    "output": """
<!DOCTYPE html>
<html>
<head><title>Coin Flip</title></head>
<body>
  <h1>Coin Flip Game</h1>
  <button onclick="flipCoin()">Flip Coin</button>
  <p id="result"></p>
  <script>
    function flipCoin() {
      const result = Math.random() < 0.5 ? 'Heads' : 'Tails';
      document.getElementById('result').textContent = 'Result: ' + result;
    }
  </script>
</body>
</html>
        """
},
{
    "question": "Create a dice roll game. Show two dice images that roll on click, with simple animation. Show the winner based on the dice values.",
    "context": "A web-based dice game using HTML, CSS, and JavaScript. Two players each roll a die. Animate the dice using CSS, and determine winner based on the higher number.",
    "output": """
<!DOCTYPE html>
<html>
<head>
  <title>Dice Roll Game</title>
  <style>
    body {
      font-family: sans-serif;
      text-align: center;
      padding: 30px;
    }
    .dice {
      font-size: 100px;
      animation: roll 0.5s ease-in-out;
    }
    @keyframes roll {
      0% { transform: rotate(0deg); }
      50% { transform: rotate(180deg); }
      100% { transform: rotate(360deg); }
    }
    button {
      padding: 10px 20px;
      margin-top: 20px;
      font-size: 16px;
    }
  </style>
</head>
<body>
  <h1>Dice Roll Game</h1>
  <div id="player1" class="dice">🎲</div>
  <div id="player2" class="dice">🎲</div>
  <button onclick="rollDice()">Roll</button>
  <h2 id="result"></h2>

  <script>
    function rollDice() {
      const dice = ['⚀','⚁','⚂','⚃','⚄','⚅'];
      const roll1 = Math.floor(Math.random()*6);
      const roll2 = Math.floor(Math.random()*6);

      document.getElementById("player1").textContent = dice[roll1];
      document.getElementById("player2").textContent = dice[roll2];

      const resultText = roll1 > roll2 ? "Player 1 Wins!" : roll1 < roll2 ? "Player 2 Wins!" : "It's a Draw!";
      document.getElementById("result").textContent = resultText;
    }
  </script>
</body>
</html>
    """
},
{
    "question": "Create a reaction time game where the user clicks a colored box as fast as possible when it changes color.",
    "context": "Web-based reaction game with CSS animations and JavaScript timers. A colored box changes randomly after 2–5 seconds. The player must click as fast as possible after the color changes. Show reaction time in ms.",
    "output": """
<!DOCTYPE html>
<html>
<head>
  <title>Reaction Time Game</title>
  <style>
    #box {
      width: 200px;
      height: 200px;
      background-color: red;
      margin: 50px auto;
      display: none;
      transition: background-color 0.2s ease;
    }
    button {
      display: block;
      margin: 20px auto;
      padding: 10px 20px;
    }
  </style>
</head>
<body>
  <h1>Reaction Time Game</h1>
  <p id="message">Click start to begin!</p>
  <div id="box" onclick="recordReaction()"></div>
  <button onclick="startGame()">Start</button>

  <script>
    let start, timeout;

    function startGame() {
      document.getElementById("box").style.display = "none";
      document.getElementById("message").textContent = "Wait for green...";
      const delay = Math.random() * 3000 + 2000;

      timeout = setTimeout(() => {
        document.getElementById("box").style.backgroundColor = "green";
        document.getElementById("box").style.display = "block";
        start = new Date().getTime();
      }, delay);
    }

    function recordReaction() {
      const end = new Date().getTime();
      const reactionTime = end - start;
      document.getElementById("message").textContent = "Your time: " + reactionTime + " ms";
      document.getElementById("box").style.display = "none";
      clearTimeout(timeout);
    }
  </script>
</body>
</html>
    """
}
]


In [ ]:
result = """
<!DOCTYPE html>
<html>
<head>
  <title>Rock Paper Scissors</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      text-align: center;
      padding: 20px;
    }
    #game-container {
      margin-top: 30px;
    }
    .score {
      font-size: 24px;
      margin-bottom: 10px;
    }
    .choice {
      display: inline-block;
      margin: 0 10px;
      padding: 10px 20px;
      cursor: pointer;
    }
    .result {
      font-size: 20px;
      margin-top: 20px;
    }
  </style>
</head>
<body>
  <h1>Rock Paper Scissors</h1>
  <div id="game-container">
    <div class="score">Player: 0 - Computer: 0</div>
    <div class="choice" data-choice="✊">Rock</div>
    <div class="choice" data-choice="✋">Paper</div>
    <div class="choice" data-choice="✌️">Scissors</div>
  </div>
  <div class="result" id="result"></div>

  <script>
    const choices = ['✊', '✋', '✌️'];
    const choiceElements = document.querySelectorAll('.choice');
    let playerScore = 0;
    let computerScore = 0;

    choiceElements.forEach(choice => {
      choice.addEventListener('click', () => {
        const playerChoice = choice.dataset.choice;
        playRound(playerChoice);
      });
    });

    function playRound(playerChoice) {
      const computerChoice = getComputerChoice();
      document.querySelector('.result').textContent = `You chose ${playerChoice} | ${getResult(playerChoice, computerChoice)} | Computer chose ${computerChoice}`;

      updateScore(playerChoice, computerChoice);
    }

    function getComputerChoice() {
      return choices[Math.floor(Math.random() * choices.length)];
    }

    function getResult(player, computer) {
      if (player === computer) return "Draw";
      if ((player === '✊' && computer === '✋') || (player === '✋' && computer === '✌️') || (player === '✌️' && computer === '✊')) return "Win";
      return "Lose";
    }

    function updateScore(player, computer) {
      const playerElement = document.querySelector(`[data-choice="${player}"]`);
      const computerElement = document.querySelector(`[data-choice="${computer}"]`);

      if (getResult(player, computer) === "Win") {
        playerScore++;
      } else if (getResult(player, computer) === "Lose") {
        computerScore++;
      }

      document.querySelector('.score').textContent = `Player: ${playerScore} - Computer: ${computerScore}`;
      playerElement.classList.add('highlight');
      computerElement.classList.remove('highlight');

      setTimeout(() => {
        playerElement.classList.remove('highlight');
      }, 500);
    }
  </script>
</body>
</html>
"""

# Konteks dan instruksi perubahan
context = "Improve user interaction with better styling and responsiveness"
question = "Make the background theme black. Make the text white. Make the button have a border radius, with a white border color and a black background. Add hover effects to buttons: change background color to white, text to white, and slightly scale the button on hover."

# Example tetap bisa pakai contoh 'Coin Flip Game' atau serupa
refined_code = refine_output_with_fewshot(result, context, question, examples)
print(refined_code)


<!DOCTYPE html>
<html>
<head>
  <title>Rock Paper Scissors</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      text-align: center;
      padding: 20px;
      background-color: #121212;
      color: white;
    }
    #game-container {
      margin-top: 30px;
    }
    .score {
      font-size: 24px;
      margin-bottom: 10px;
    }
    .choice {
      display: inline-block;
      margin: 0 10px;
      padding: 10px 20px;
      cursor: pointer;
      border-radius: 5px;
      border: 2px solid white;
      background-color: #333333;
      transition: background-color 0.3s, transform 0.15s;
    }
    .choice:hover {
      background-color: white;
      color: #333333;
      transform: scale(1.02);
    }
    .result {
      font-size: 20px;
      margin-top: 20px;
    }
  </style>
</head>
<body>
  <h1>Rock Paper Scissors</h1>
  <div id="game-container">
    <div class="score">Player: 0 - Computer: 0</div>
    <div class="choice" data-choice="✊">Rock</div>
    <div class

```
<!DOCTYPE html>
<html>
<head>
  <title>Rock Paper Scissors</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      text-align: center;
      padding: 20px;
      background-color: #121212;
      color: white;
    }
    #game-container {
      margin-top: 30px;
    }
    .score {
      font-size: 24px;
      margin-bottom: 10px;
    }
    .choice {
      display: inline-block;
      margin: 0 10px;
      padding: 10px 20px;
      cursor: pointer;
      border-radius: 5px;
      border: 2px solid white;
      background-color: #333333;
      transition: background-color 0.3s, transform 0.15s;
    }
    .choice:hover {
      background-color: white;
      color: #333333;
      transform: scale(1.02);
    }
    .result {
      font-size: 20px;
      margin-top: 20px;
    }
  </style>
</head>
<body>
  <h1>Rock Paper Scissors</h1>
  <div id="game-container">
    <div class="score">Player: 0 - Computer: 0</div>
    <div class="choice" data-choice="✊">Rock</div>
    <div class="choice" data-choice="✋">Paper</div>
    <div class="choice" data-choice="✌️">Scissors</div>
  </div>
  <div class="result" id="result"></div>

  <script>
    const choices = ['✊', '✋', '✌️'];
    const choiceElements = document.querySelectorAll('.choice');
    let playerScore = 0;
    let computerScore = 0;

    choiceElements.forEach(choice => {
      choice.addEventListener('click', () => {
        const playerChoice = choice.dataset.choice;
        playRound(playerChoice);
      });
    });

    function playRound(playerChoice) {
      const computerChoice = getComputerChoice();
      document.querySelector('.result').textContent = `You chose ${playerChoice} | ${getResult(playerChoice, computerChoice)} | Computer chose ${computerChoice}`;

      updateScore(playerChoice, computerChoice);
    }

    function getComputerChoice() {
      return choices[Math.floor(Math.random() * choices.length)];
    }

    function getResult(player, computer) {
      if (player === computer) return "Draw";
      if ((player === '✊' && computer === '✋') || (player === '✋' && computer === '✌️') || (player === '✌️' && computer === '✊')) return "Win";
      return "Lose";
    }

    function updateScore(player, computer) {
      const playerElement = document.querySelector(`[data-choice="${player}"]`);
      const computerElement = document.querySelector(`[data-choice="${computer}"]`);

      if (getResult(player, computer) === "Win") {
        playerScore++;
      } else if (getResult(player, computer) === "Lose") {
        computerScore++;
      }

      document.querySelector('.score').textContent = `Player: ${playerScore} - Computer: ${computerScore}`;
      playerElement.classList.add('highlight');
      computerElement.classList.remove('highlight');

      setTimeout(() => {
        playerElement.classList.remove('highlight');
      }, 500);
    }
  </script>
</body>
</html>
```

In [ ]:
result2 = """
<!DOCTYPE html>
<html>
<head>
  <title>Rock Paper Scissors</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      text-align: center;
      padding: 20px;
      background-color: #121212;
      color: white;
    }
    #game-container {
      margin-top: 30px;
    }
    .score {
      font-size: 24px;
      margin-bottom: 10px;
    }
    .choice {
      display: inline-block;
      margin: 0 10px;
      padding: 10px 20px;
      cursor: pointer;
      border-radius: 5px;
      border: 2px solid white;
      background-color: #333333;
      transition: background-color 0.3s, transform 0.15s;
    }
    .choice:hover {
      background-color: white;
      color: #333333;
      transform: scale(1.02);
    }
    .result {
      font-size: 20px;
      margin-top: 20px;
    }
  </style>
</head>
<body>
  <h1>Rock Paper Scissors</h1>
  <div id="game-container">
    <div class="score">Player: 0 - Computer: 0</div>
    <div class="choice" data-choice="✊">Rock</div>
    <div class="choice" data-choice="✋">Paper</div>
    <div class="choice" data-choice="✌️">Scissors</div>
  </div>
  <div class="result" id="result"></div>

  <script>
    const choices = ['✊', '✋', '✌️'];
    const choiceElements = document.querySelectorAll('.choice');
    let playerScore = 0;
    let computerScore = 0;

    choiceElements.forEach(choice => {
      choice.addEventListener('click', () => {
        const playerChoice = choice.dataset.choice;
        playRound(playerChoice);
      });
    });

    function playRound(playerChoice) {
      const computerChoice = getComputerChoice();
      document.querySelector('.result').textContent = `You chose ${playerChoice} | ${getResult(playerChoice, computerChoice)} | Computer chose ${computerChoice}`;

      updateScore(playerChoice, computerChoice);
    }

    function getComputerChoice() {
      return choices[Math.floor(Math.random() * choices.length)];
    }

    function getResult(player, computer) {
      if (player === computer) return "Draw";
      if ((player === '✊' && computer === '✋') || (player === '✋' && computer === '✌️') || (player === '✌️' && computer === '✊')) return "Win";
      return "Lose";
    }

    function updateScore(player, computer) {
      const playerElement = document.querySelector(`[data-choice="${player}"]`);
      const computerElement = document.querySelector(`[data-choice="${computer}"]`);

      if (getResult(player, computer) === "Win") {
        playerScore++;
      } else if (getResult(player, computer) === "Lose") {
        computerScore++;
      }

      document.querySelector('.score').textContent = `Player: ${playerScore} - Computer: ${computerScore}`;
      playerElement.classList.add('highlight');
      computerElement.classList.remove('highlight');

      setTimeout(() => {
        playerElement.classList.remove('highlight');
      }, 500);
    }
  </script>
</body>
</html>
"""

# Konteks dan instruksi perubahan
context2 = "Enhance the visual design and layout of the Rock Paper Scissors game. Apply modern web design elements and organize the content for better readability and user experience"
question2 = """Integrate FontAwesome icons for the Rock, Paper, and Scissors buttons using the following icons:
- <i class="fa-regular fa-hand-back-fist"></i> for Rock
- <i class="fa-regular fa-hand"></i> for Paper
- <i class="fa-regular fa-hand-scissors"></i> for Scissors

In addition:
- Wrap all content (header, buttons, result text, score) inside a centered card with a glassmorphism effect (blurred translucent background).
- Full background still black
- Ensure the layout is vertically centered and responsive.
- Link the FontAwesome CDN inside the <head> tag.
"""

# Example tetap bisa pakai contoh 'Coin Flip Game' atau serupa
refined_code2 = refine_output_with_fewshot(
    result=result2,
    context=context2,
    question=question2,
    examples=examples
)

print(refined_code2)


<!DOCTYPE html>
<html>
<head>
  <title>Rock Paper Scissors</title>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
  <style>
    body {
      font-family: Arial, sans-serif;
      background-color: #121212;
      color: white;
      display: flex;
      justify-content: center;
      align-items: center;
      min-height: 100vh;
      margin: 0;
    }
    .card {
      background-color: rgba(255, 255, 255, 0.2);
      border-radius: 10px;
      box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
      padding: 20px;
      text-align: center;
      width: 300px;
    }
    .card h1 {
      margin-bottom: 20px;
    }
    .score {
      font-size: 24px;
      margin-bottom: 10px;
    }
    .choice {
      display: inline-block;
      margin: 0 10px;
      padding: 10px 20px;
      cursor: pointer;
      border-radius: 5px;
      border: 2px solid white;
      background-color: #333333;
      transition: background-color 0.3s, transform 0.15s

```
<!DOCTYPE html>
<html>
<head>
  <title>Rock Paper Scissors</title>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
  <style>
    body {
      font-family: Arial, sans-serif;
      background-color: #121212;
      color: white;
      display: flex;
      justify-content: center;
      align-items: center;
      min-height: 100vh;
      margin: 0;
    }
    .card {
      background-color: rgba(255, 255, 255, 0.2);
      border-radius: 10px;
      box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
      padding: 20px;
      text-align: center;
      width: 300px;
    }
    .card h1 {
      margin-bottom: 20px;
    }
    .score {
      font-size: 24px;
      margin-bottom: 10px;
    }
    .choice {
      display: inline-block;
      margin: 0 10px;
      padding: 10px 20px;
      cursor: pointer;
      border-radius: 5px;
      border: 2px solid white;
      background-color: #333333;
      transition: background-color 0.3s, transform 0.15s;
    }
    .choice:hover {
      background-color: white;
      color: #333333;
      transform: scale(1.02);
    }
    .choice i {
      margin-right: 5px;
    }
    .result {
      font-size: 20px;
      margin-top: 20px;
    }
  </style>
</head>
<body>
  <div class="card">
    <h1>Rock Paper Scissors</h1>
    <div class="score">Player: 0 - Computer: 0</div>
    <div class="choice" data-choice="✊" onclick="choose('✊')">
      <i class="fa-regular fa-hand-back-fist"></i> Rock
    </div>
    <div class="choice" data-choice="✋" onclick="choose('✋')">
      <i class="fa-regular fa-hand"></i> Paper
    </div>
    <div class="choice" data-choice="✌️" onclick="choose('✌️')">
      <i class="fa-regular fa-hand-scissors"></i> Scissors
    </div>
    <div class="result" id="result"></div>
  </div>

  <script>
    function choose(playerChoice) {
      const choices = ['✊', '✋', '✌️'];
      const computerChoice = choices[Math.floor(Math.random() * 3)];
      const resultText = getResult(playerChoice, computerChoice);
      document.getElementById('result').textContent = resultText;
      updateScore(playerChoice, computerChoice);
    }

    function getResult(player, computer) {
      if (player === computer) return "It's a tie!";
      if (
        (player === '✊' && computer === '✋') ||
        (player === '✋' && computer === '✌️') ||
        (player === '✌️' && computer === '✊')
      ) {
        return 'Player wins!';
      } else {
        return 'Computer wins!';
      }
    }

    function updateScore(player, computer) {
      const playerScore = parseInt(document.querySelector('.score').textContent.split('-')[0].split(' ')[1]);
      const computerScore = parseInt(document.querySelector('.score').textContent.split('-')[1].split(' ')[0]);

      if (player === computer) {
        document.querySelector('.score').textContent = `Player: ${playerScore} - Computer: ${computerScore}`;
      } else {
        document.querySelector('.score').textContent = `Player: ${playerScore + (player === '✊' || player === '✋' || player === '✌️' ? 1 : 0)} - Computer: ${computerScore + (computer === '✊' || computer === '✋' || computer === '✌️' ? 1 : 0)}`;
      }
    }
  </script>
</body>
```